In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ishworsubedii/nepali-speech-to-text-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.01G/1.01G [00:09<00:00, 119MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ishworsubedii/nepali-speech-to-text-dataset/versions/1


In [ ]:
!mv /root/.cache/kagglehub/datasets/ishworsubedii/nepali-speech-to-text-dataset/versions/1 /content/drive/MyDrive

In [ ]:
!wget https://huggingface.co/datasets/spktsagar/openslr-nepali-asr-cleaned/resolve/main/openslr-nepali-asr-cleaned.py

Could not load dataset directly: Dataset scripts are no longer supported, but found openslr-nepali-asr-cleaned.py
Attempting to load from local path if available...


In [45]:
!pip install -q datasets accelerate peft bitsandbytes jiwer evaluate librosa soundfile


In [3]:
from datasets import load_dataset, concatenate_datasets

nepali = load_dataset("amitpant7/nepali-speech-to-text")



Repo card metadata block was not found. Setting CardData to empty.


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="multilingual", task="transcribe")

def prepare(example):
    audio = example["audio"]
    example["input_features"] = processor(audio["array"], sampling_rate=16000).input_features[0]
    example["labels"] = processor.tokenizer(example["transcription"]).input_ids # Corrected from example["sentence"]
    return example

dataset = nepali.map(prepare, remove_columns=['audio', 'transcription'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [1]:
!apt-get update && apt-get install -y ffmpeg  # Fix for TorchCodec/FFmpeg issues
!pip install -q datasets==2.21.0 accelerate peft bitsandbytes jiwer evaluate librosa soundfile transformers

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,086 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,827 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [

In [2]:
from datasets import load_dataset, concatenate_datasets

# Load the dataset (it has multiple 'train.*' splits)
nepali = load_dataset("amitpant7/nepali-speech-to-text")

# Concatenate all train splits (ignore any test if present)
train_splits = [split for split in nepali if 'train' in split]
nepali_train = concatenate_datasets([nepali[split] for split in train_splits])

print(f"Loaded {len(nepali_train)} training samples.")
print(nepali_train)
print(nepali_train[0])  # Preview one example

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Loaded 2650 training samples.
Dataset({
    features: ['audio', 'transcription'],
    num_rows: 2650
})
{'audio': {'path': 'train_audio_0.wav', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        2.60547258e-05,  8.18362423e-06, -4.72624306e-05]), 'sampling_rate': 32000}, 'transcription': 'म पनि जान्छु है त अहिले लाई ।'}


In [15]:
from transformers import WhisperProcessor
import torch

# Load processor with Nepali language
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    language="ne",
    task="transcribe"
)

def prepare(example):
    audio = example["audio"]
    # Resample audio to 16000 Hz
    audio = audio.copy_resample(16000)
    # Use actual sampling rate for resampling; return PyTorch tensors
    example["input_features"] = processor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_tensors="pt"
    ).input_features[0]

    # Encode transcription to labels
    example["labels"] = processor.tokenizer(example["transcription"]).input_ids
    return example

# Map safely: single process, non-batched to avoid OOM
nepali_train = nepali_train.map(
    prepare,
    remove_columns=nepali_train.column_names,  # Clean up after processing
    num_proc=1,
    batched=False
)

# Optional: Add a simple validation split (10% of train)
nepali_train = nepali_train.train_test_split(test_size=0.1, shuffle=True)
print("Processed dataset splits:", nepali_train)

# Save to disk if needed (for resuming)
nepali_train.save_to_disk("/content/drive/MyDrive/nepali_whisper_processed")

ValueError: Column to remove ['train', 'test'] not in the dataset. Current columns in the dataset: ['input_features', 'labels']

In [11]:
!pip install --upgrade transformers


In [46]:
from datasets import load_from_disk
import torch
from transformers import WhisperProcessor
from peft import LoraConfig, get_peft_model, TaskType

# Load saved dataset (includes train/test splits)
nepali_train = load_from_disk("/content/drive/MyDrive/nepali_whisper_processed")
print("Loaded dataset:", nepali_train)
print(f"Train samples: {len(nepali_train['train'])}, Eval samples: {len(nepali_train['test'])}")

# Reload processor (for collator and metrics)
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small",
    language="ne",
    task="transcribe"
)

# Your compute_metrics function (from previous cell)
import evaluate
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Loaded dataset: DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2385
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 265
    })
})
Train samples: 2385, Eval samples: 265


In [49]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# -----------------------------
# CORRECTED Data collator for Whisper
# -----------------------------
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # input_features already processed, just stack them
        input_features = [torch.tensor(f["input_features"]) for f in features]

        # Pad input features to same length
        input_features = torch.nn.utils.rnn.pad_sequence(
            input_features, batch_first=True, padding_value=0.0
        )

        # Handle labels
        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch = {
            "input_features": input_features,
            "labels": labels,
        }
        return batch

# -----------------------------
# Training parameters
# -----------------------------
model_id = "small"
task = "transcribe"
language = "ne"

# LoRA parameters
r = 16
lora_alpha = 32
lora_dropout = 0.1

# Training parameters
learning_rate = 1e-4
train_batch_size = 8  # Reduced to avoid memory issues
gradient_accumulation_steps = 2
fp16 = True
warmup_steps = 100
max_steps = 1000
save_steps = 200
logging_steps = 50

experiment_name = f"whisper-{model_id}-lora-nepali"

# -----------------------------
# Load processor and model
# -----------------------------
processor = WhisperProcessor.from_pretrained(
    f"openai/whisper-{model_id}",
    language=language,
    task=task
)

# Load model
model = WhisperForConditionalGeneration.from_pretrained(
    f"openai/whisper-{model_id}",
    device_map="auto"
)

# Configure model
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.config.use_cache = False
model.gradient_checkpointing_enable()

# -----------------------------
# Apply LoRA
# -----------------------------
peft_config = LoraConfig(
    r=r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=lora_dropout,
    bias="none",
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# -----------------------------
# Data collator
# -----------------------------
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# -----------------------------
# Training arguments
# -----------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir=f"/content/drive/MyDrive/{experiment_name}",
    per_device_train_batch_size=train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    max_steps=max_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    eval_steps=200,
    save_total_limit=3,
    predict_with_generate=True,
    generation_max_length=225,
    report_to=["tensorboard"],
    remove_unused_columns=False,
    fp16=fp16,
)

# -----------------------------
# Trainer
# -----------------------------
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=nepali_train["train"],
    eval_dataset=nepali_train["test"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

/tmp/ipython-input-2157445051.py:128: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


trainable params: 1,769,472 || all params: 243,504,384 || trainable%: 0.7267


You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
50,2.373200
100,1.405700
150,1.033900
200,0.932300
250,0.862900
300,0.855600
350,0.781700
400,0.750900
450,0.714600
500,0.612200


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=1000, training_loss=0.7454318752288819, metrics={'train_runtime': 3440.5565, 'train_samples_per_second': 4.65, 'train_steps_per_second': 0.291, 'total_flos': 4.6319330304e+18, 'train_loss': 0.7454318752288819, 'epoch': 6.668896321070234})

In [50]:
import evaluate
import torch

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [52]:
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_from_disk
import evaluate

# Load the dataset and model
nepali_train = load_from_disk("/content/drive/MyDrive/nepali_whisper_processed")
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="ne", task="transcribe")

# Load your trained model (use the latest checkpoint)
model_path = f"/content/drive/MyDrive/whisper-small-lora-nepali/checkpoint-1000"  # or specific checkpoint
model = WhisperForConditionalGeneration.from_pretrained(model_path)
model.eval()

# Load WER metric
wer_metric = evaluate.load("wer")

def predict_and_compare(model, processor, dataset, num_samples=10):
    """Compare model predictions with ground truth"""

    results = []
    wer_scores = []

    # Get random samples from test set
    test_dataset = dataset["test"]
    indices = np.random.choice(len(test_dataset), min(num_samples, len(test_dataset)), replace=False)

    print(" Comparing Predictions vs Ground Truth:\n")
    print("=" * 100)

    for i, idx in enumerate(indices):
        # Convert numpy index to standard integer
        sample = test_dataset[int(idx)]

        # Get input features
        input_features = torch.tensor(sample["input_features"]).unsqueeze(0)

        # Generate prediction
        with torch.no_grad():
            predicted_ids = model.generate(input_features, max_length=225)

        # Decode predictions and labels
        prediction = processor.tokenizer.decode(predicted_ids[0], skip_special_tokens=True)

        # Get ground truth (remove -100 padding)
        label_ids = sample["labels"]
        label_ids = [token_id for token_id in label_ids if token_id != -100]
        ground_truth = processor.tokenizer.decode(label_ids, skip_special_tokens=True)

        # Calculate WER for this sample
        wer = wer_metric.compute(predictions=[prediction], references=[ground_truth])
        wer_scores.append(wer)

        results.append({
            'prediction': prediction,
            'ground_truth': ground_truth,
            'wer': wer
        })

        # Print comparison
        print(f"Sample {i+1}:")
        print(f"Ground Truth: {ground_truth}")
        print(f"Prediction:   {prediction}")
        print(f"WER: {wer:.2%}")
        print("-" * 80)

    # Overall statistics
    avg_wer = np.mean(wer_scores)
    print(f"\nOverall Average WER: {avg_wer:.2%}")
    print(f"WER Range: {np.min(wer_scores):.2%} - {np.max(wer_scores):.2%}")

    return results, wer_scores

# Run prediction comparison
results, wer_scores = predict_and_compare(model, processor, nepali_train, num_samples=10)

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


 Comparing Predictions vs Ground Truth:



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Sample 1:
Ground Truth: साथी तिमी बेनी बजार गएको छौ?
Prediction:   साति तिमी बेनी बजार गयागो छुु ।
WER: 66.67%
--------------------------------------------------------------------------------
Sample 2:
Ground Truth: काम र पढाइले गर्दा नै हो धेरै त ।
Prediction:    काम्रा पडाँ ब्राँ ब्राँ ब्राँ ग्र्दा नेँ बुदिरिता ।
WER: 88.89%
--------------------------------------------------------------------------------
Sample 3:
Ground Truth: अनाज अन्न र ओरिजा सातिभाको बीउ
Prediction:   अनाज अन्न्ड र उरिजा सातिबहाको व्यु
WER: 66.67%
--------------------------------------------------------------------------------
Sample 4:
Ground Truth: पोखरा त नेपालकै सफा सहर हो नि,सायद धुलो कम नै होला ।
Prediction:   पुख्रात नेपालके सफा सहरहो नि साया दुलुकम्ने अला ।
WER: 83.33%
--------------------------------------------------------------------------------
Sample 5:
Ground Truth: सोडियम क्लोराइडको खनिज अवस्था
Prediction:   सोडियम क्लोराइटको खानिज अब्ष्टा
WER: 75.00%
-----------------------------------------------

Original model before finetuning

In [55]:
import evaluate
import torch
import numpy as np # Import numpy

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

def compare_with_base_model(num_samples=5):
    """Compare your fine-tuned model with the base Whisper model"""

    # Load base model
    base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
    base_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
    base_model.eval()

    test_dataset = nepali_train["test"]
    indices = np.random.choice(len(test_dataset), min(num_samples, len(test_dataset)), replace=False)

    print(" Comparison: Fine-tuned vs Base Model")
    print("=" * 100)

    for i, idx in enumerate(indices):
        # Convert numpy index to standard integer
        sample = test_dataset[int(idx)] # Corrected here
        input_features = torch.tensor(sample["input_features"]).unsqueeze(0)

        # Your fine-tuned model prediction
        with torch.no_grad():
            fine_tuned_pred = model.generate(input_features, max_length=225)
        ft_prediction = processor.tokenizer.decode(fine_tuned_pred[0], skip_special_tokens=True)

        # Base model prediction
        with torch.no_grad():
            base_pred = base_model.generate(input_features, max_length=225)
        base_prediction = base_processor.tokenizer.decode(base_pred[0], skip_special_tokens=True)

        # Ground truth
        label_ids = sample["labels"]
        label_ids = [token_id for token_id in label_ids if token_id != -100]
        ground_truth = processor.tokenizer.decode(label_ids, skip_special_tokens=True)

        # Calculate WER
        ft_wer = wer_metric.compute(predictions=[ft_prediction], references=[ground_truth])
        base_wer = wer_metric.compute(predictions=[base_prediction], references=[ground_truth])

        print(f"Sample {i+1}:")
        print(f"Ground Truth: {ground_truth}")
        print(f"Fine-tuned:   {ft_prediction} (WER: {ft_wer:.2%})")
        print(f"Base Model:   {base_prediction} (WER: {base_wer:.2%})")
        # Avoid division by zero if base_wer is 0
        improvement = ((base_wer - ft_wer) / base_wer * 100) if base_wer != 0 else float('inf')
        print(f"Improvement:  {improvement:+.1f}%")
        print("-" * 80)

# Uncomment to run comparison
compare_with_base_model(num_samples=3)

 Comparison: Fine-tuned vs Base Model
Sample 1:
Ground Truth: यो गाउँको सिमाना पातेमेला जामनेपानी छाम कर्णाली नदीसँग जोडिएको छ
Fine-tuned:   यो गएउगाउँगो सिमाना पाटे मेला जामणे पानि चाम् कर्नाली नदी सग्दिएको छ (WER: 90.00%)
Base Model:    यो गवको शिमाना पाते मेला जामने पानी चाम करनाली नदी संगा जोडी एको चाए. (WER: 130.00%)
Improvement:  +30.8%
--------------------------------------------------------------------------------
Sample 2:
Ground Truth: उहाँलाई चमत्कारिक व्यक्तित्वको रूपमा लिने गरिन्छ
Fine-tuned:   वाहालाई चम्ब्ट कारिक बेक्तिक्तोको रूपमा लिने गरिन्छ (WER: 66.67%)
Base Model:    वहा ले चवमत करिक बेक्तिख तोको रुप माल दिने गरिन सा. (WER: 183.33%)
Improvement:  +63.6%
--------------------------------------------------------------------------------
Sample 3:
Ground Truth: वालग्रिन डट् कम् यसको सुरुवातका वर्षमै प्रगतिको बाटामा लागेको थियो
Fine-tuned:   वाल्ग्रिन्ड ड्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्ट्

For enlish language

In [57]:
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Load your fine-tuned model
model_path = f"/content/drive/MyDrive/whisper-small-lora-nepali/checkpoint-1000"
model = WhisperForConditionalGeneration.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# Test English sentences
english_tests = [
    "Hello, how are you doing today?",
    "The weather is beautiful outside.",
    "I would like to order a coffee please.",
]

print("🧪 Testing with Language Forcing")
print("=" * 60)

for i, text in enumerate(english_tests, 1):
    dummy_audio = np.random.randn(16000 * 3)

    input_features = processor(
        dummy_audio,
        sampling_rate=16000,
        return_tensors="pt"
    ).input_features

    # Force English language during generation
    forced_decoder_ids = processor.get_decoder_prompt_ids(
        language="en",
        task="transcribe"
    )

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            max_length=225,
            forced_decoder_ids=forced_decoder_ids  # Force English output
        )

    prediction = processor.decode(predicted_ids[0], skip_special_tokens=True)

    print(f"{i}. Expected: '{text}'")
    print(f"   Got:      '{prediction}'")
    print()

🧪 Testing with Language Forcing
1. Expected: 'Hello, how are you doing today?'
   Got:      ' Ssshh!'

2. Expected: 'The weather is beautiful outside.'
   Got:      ' Ssshh!'

3. Expected: 'I would like to order a coffee please.'
   Got:      '.'



Model completely forgots english language. LOL